# Transfer Learning

Using the high level transfer learning APIs, you can easily customize pretrained models for feature extraction or fine-tuning. 

In this notebook, we will use a pre-trained Inception_V1 model. But we will operate on the pre-trained model to freeze first few layers, replace the classifier on the top, then fine tune the whole model. And we use the fine-tuned model to solve the dogs-vs-cats classification problem,

## Preparation

### 1. Get the dogs-vs-cats datasets

Download the training dataset from https://www.kaggle.com/c/dogs-vs-cats and extract it. 

The following commands copy about 1100 images of cats and dogs into demo/cats and demo/dogs separately. 
```shell
mkdir -p demo/dogs
mkdir -p demo/cats
cp train/cat.7* demo/cats
cp train/dog.7* demo/dogs```

### 2. Get the pre-trained Inception-V1 model

Download the pre-trained Inception-V1 model from [Zoo](https://s3-ap-southeast-1.amazonaws.com/bigdl-models/imageclassification/imagenet/bigdl_inception-v1_imagenet_0.4.0.model) 
 Alternatively, user may also download pre-trained caffe/Tensorflow/keras model.

In [1]:
import re

from bigdl.nn.criterion import CrossEntropyCriterion
from pyspark import SparkConf
from pyspark.ml import Pipeline
from pyspark.sql.functions import col, udf
from pyspark.sql.types import DoubleType, StringType

from zoo.common.nncontext import *
from zoo.feature.image import *
from zoo.pipeline.api.keras.layers import Dense, Input, Flatten
from zoo.pipeline.api.keras.models import *
from zoo.pipeline.api.net import *
from zoo.pipeline.nnframes import *

creating: createDefault
creating: createSGD
creating: createSeqToTensor
creating: createSeqToTensor
creating: createSeqToTensor
creating: createSeqToTensor
creating: createSeqToTensor


In [2]:
sparkConf = SparkConf().setAppName("ImageTransferLearningExample")
sc = init_nncontext(sparkConf)

manually set model_path and image_path for training

1. model_path = path to the pre-trained models. (E.g. path/to/model/bigdl_inception-v1_imagenet_0.4.0.model)

2. image_path = path to the folder of the training images. (E.g. path/to/data/dogs-vs-cats/demo/)

In [3]:
model_path = "path/to/model/bigdl_inception-v1_imagenet_0.4.0.model"
image_path = "path/to/data/dogs-vs-cats/demo/*/*"
imageDF = NNImageReader.readImages(image_path, sc)

In [4]:
getName = udf(lambda row:
                  re.search(r'(cat|dog)\.([\d]*)\.jpg', row[0], re.IGNORECASE).group(0),
                  StringType())
getLabel = udf(lambda name: 1.0 if name.startswith('cat') else 2.0, DoubleType())

labelDF = imageDF.withColumn("name", getName(col("image"))) \
        .withColumn("label", getLabel(col('name')))
(trainingDF, validationDF) = labelDF.randomSplit([0.9, 0.1])
labelDF.show()

+--------------------+------------+-----+
|               image|        name|label|
+--------------------+------------+-----+
|[file:/home/xiaxu...|cat.7125.jpg|  1.0|
|[file:/home/xiaxu...|cat.7580.jpg|  1.0|
|[file:/home/xiaxu...| cat.771.jpg|  1.0|
|[file:/home/xiaxu...|cat.7701.jpg|  1.0|
|[file:/home/xiaxu...|cat.7232.jpg|  1.0|
|[file:/home/xiaxu...|cat.7901.jpg|  1.0|
|[file:/home/xiaxu...|cat.7295.jpg|  1.0|
|[file:/home/xiaxu...|cat.7250.jpg|  1.0|
|[file:/home/xiaxu...|cat.7782.jpg|  1.0|
|[file:/home/xiaxu...|cat.7273.jpg|  1.0|
|[file:/home/xiaxu...|cat.7076.jpg|  1.0|
|[file:/home/xiaxu...|cat.7886.jpg|  1.0|
|[file:/home/xiaxu...|cat.7279.jpg|  1.0|
|[file:/home/xiaxu...|cat.7364.jpg|  1.0|
|[file:/home/xiaxu...|cat.7079.jpg|  1.0|
|[file:/home/xiaxu...|cat.7957.jpg|  1.0|
|[file:/home/xiaxu...|cat.7566.jpg|  1.0|
|[file:/home/xiaxu...|cat.7370.jpg|  1.0|
|[file:/home/xiaxu...|cat.7596.jpg|  1.0|
|[file:/home/xiaxu...|cat.7061.jpg|  1.0|
+--------------------+------------

## Fine-tune a pre-trained model

We compose a pipeline that includes feature transform, pretrained model and Logistic Regression.

In [5]:
transformer = ChainedPreprocessing(
        [RowToImageFeature(), ImageResize(256, 256), ImageCenterCrop(224, 224),
         ImageChannelNormalize(123.0, 117.0, 104.0), ImageMatToTensor(), ImageFeatureToTensor()])

creating: createRowToImageFeature
creating: createImageResize
creating: createImageCenterCrop
creating: createImageChannelNormalize
creating: createImageMatToTensor
creating: createImageFeatureToTensor
creating: createChainedPreprocessing


### Load a pre-trained model

We use the Net API to load a pre-trained model, including models saved by Analytics Zoo, BigDL, Torch, Caffe and Tensorflow. Please refer to [Net API Guide](https://analytics-zoo.github.io/master/#APIGuide/PipelineAPI/net/).

In [6]:
full_model = Net.load_bigdl(model_path)

### Remove the last few layers

When a model is loaded using Net, we can use the newGraph(output) api to define a Model with the output specified by the parameter. 

In [7]:
model = full_model.new_graph(["pool5/drop_7x7_s1"])

The returning model's output layer is "pool5/drop_7x7_s1".

### Freeze some layers

freeze layers from input to pool4/3x3_s2 inclusive

In [8]:
model.freeze_up_to(["pool4/3x3_s2"])

### Add a few new layers

In [9]:
inputNode = Input(name="input", shape=(3, 224, 224))
inception = model.to_keras()(inputNode)
flatten = Flatten()(inception)
logits = Dense(2)(flatten)
lrModel = Model(inputNode, logits)
classifier = NNClassifier(lrModel, CrossEntropyCriterion(), transformer) \
        .setLearningRate(0.003).setBatchSize(40).setMaxEpoch(1).setFeaturesCol("image") \
        .setCachingSample(False)
pipeline = Pipeline(stages=[classifier])

creating: createZooKerasInput
creating: createZooKerasFlatten
creating: createZooKerasDense
creating: createZooKerasModel
creating: createCrossEntropyCriterion
creating: createScalarToTensor
creating: createFeatureLabelPreprocessing
creating: createNNClassifier


# Train the model

The transfer learning can finish in a few minutes. 

In [10]:
catdogModel = pipeline.fit(trainingDF)
predictionDF = catdogModel.transform(validationDF).cache()
predictionDF.show()

creating: createToTuple
creating: createChainedPreprocessing
creating: createTensorToSample
creating: createChainedPreprocessing
+--------------------+------------+-----+----------+
|               image|        name|label|prediction|
+--------------------+------------+-----+----------+
|[file:/home/xiaxu...|cat.7013.jpg|  1.0|       1.0|
|[file:/home/xiaxu...|cat.7019.jpg|  1.0|       1.0|
|[file:/home/xiaxu...|cat.7035.jpg|  1.0|       1.0|
|[file:/home/xiaxu...|cat.7038.jpg|  1.0|       1.0|
|[file:/home/xiaxu...|cat.7039.jpg|  1.0|       1.0|
|[file:/home/xiaxu...|cat.7044.jpg|  1.0|       1.0|
|[file:/home/xiaxu...|cat.7046.jpg|  1.0|       1.0|
|[file:/home/xiaxu...|cat.7049.jpg|  1.0|       1.0|
|[file:/home/xiaxu...|cat.7071.jpg|  1.0|       1.0|
|[file:/home/xiaxu...|cat.7072.jpg|  1.0|       1.0|
|[file:/home/xiaxu...|cat.7076.jpg|  1.0|       1.0|
|[file:/home/xiaxu...|cat.7077.jpg|  1.0|       1.0|
|[file:/home/xiaxu...|cat.7096.jpg|  1.0|       1.0|
|[file:/home/xiaxu...|c

In [11]:
correct = predictionDF.filter("label=prediction").count()
overall = predictionDF.count()
accuracy = correct * 1.0 / overall
print("Test Error = %g " % (1.0 - accuracy))

Test Error = 0.0316742 


As we can see, the model from transfer learning can achieve over 95% accuracy on the validation set.